In [7]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import joblib
import os

# Function to fit ARIMA model and make predictions
def fit_arima(train_data, order):
    model = ARIMA(train_data, order=order)
    model_fit = model.fit()
    return model_fit

# Function to train ARIMA models and save them to files
def train_and_save_arima_models(dataframe, p_d_q_values, output_directory):
    trained_models = {}

    for ticker_symbol, order in p_d_q_values.items():
        data = dataframe['Close'].values
        trained_model = fit_arima(data, order)
        trained_models[ticker_symbol] = trained_model

        # Save the trained model to a file in the specified output directory
        model_filename = os.path.join(output_directory, f'arima_model_{ticker_symbol}.joblib')
        joblib.dump(trained_model, model_filename)

    return trained_models

# Modified data preprocessing function that includes ARIMA model training
def ARIMA_and_RL_Agent_Preprocess(tickers, start_date, p_d_q_values, output_directory):
    filtered_dfs = {}  # Store filtered DataFrames for each ticker
    trained_models = {}  # Store trained ARIMA models for each ticker

    for ticker in tickers:
        # Read the data CSV for the current ticker
        df = pd.read_csv(f"{ticker}_Daily_Data.csv")

        # Convert the 'Date' column to datetime if it's not already
        df['Date'] = pd.to_datetime(df['Date'])

        # Filter the DataFrame to keep rows from the start_date onwards
        filtered_df = df[df['Date'] >= start_date]

        # Reset the index of the filtered DataFrame
        filtered_df.reset_index(drop=True, inplace=True)

        # Store the filtered DataFrame in the dictionary
        filtered_dfs[ticker] = filtered_df

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Step 2: Train and Save ARIMA Models for Each Ticker
    for ticker in tickers:
        # Load filtered DataFrame for the current ticker
        df = filtered_dfs[ticker]
        
        # Train and save ARIMA model for the current ticker in the specified output directory
        trained_models = train_and_save_arima_models(df, {ticker: p_d_q_values[ticker]}, output_directory)

        # Access trained ARIMA model for the current ticker
        arima_model = trained_models[ticker]

    return filtered_dfs, trained_models

if __name__ == "__main__":
    # Define your list of tickers
    tickers = ["AAPL", "MSFT", "ORCL"]  # Replace with your list of tickers
    
    # Define your desired start date
    start_date = pd.to_datetime('2022-03-01')  # Change this to your desired start date
    
    # Define (p, d, q) values for each ticker
    p_d_q_values = {
        "AAPL": (1, 1, 2),
        "MSFT": (2, 1, 2),
        "ORCL": (2, 1, 2),
        # Add more tickers and their corresponding (p, d, q) values here
    }
    
    # Define the output directory within the current directory
    output_directory = "trained_models"
    
    # Step 1: Perform Data Preprocessing and Save Filtered DataFrames
    filtered_data, _ = ARIMA_and_RL_Agent_Preprocess(tickers, start_date, p_d_q_values, output_directory)


C:\Users\Shemeen62\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [2]:
import joblib
import pandas as pd
import yfinance as yf
import os

# Function to load a saved ARIMA model and make future predictions
def predict_future_prices(stock_ticker, num_days, models_directory):
    # Load the saved model for the chosen stock from the specified directory
    model_filename = os.path.join(models_directory, f'arima_model_{stock_ticker}.joblib')
    user_model = joblib.load(model_filename)

    # Fetch historical data for the chosen stock
    ticker = yf.Ticker(stock_ticker)
    df = ticker.history(interval='1d', start='2022-03-01')

    # Determine the last date in the historical data
    last_date = df.index[-1]

    # Generate date range for the next num_days days
    future_dates = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=num_days)

    # Make predictions for the user's chosen stock for the next num_days days
    future_predictions = user_model.forecast(steps=num_days)

    # Create a dictionary to store the date and predicted price
    predicted_prices = {}

    for i, date in enumerate(future_dates):
        predicted_prices[date.strftime('%Y-%m-%d')] = future_predictions[i]

    return predicted_prices

# Example usage:
if __name__ == "__main__":
    user_ticker = input("Enter a stock ticker (e.g., AAPL, MSFT, ORCL): ")
    user_days = int(input("Enter the number of future days for prediction: "))
    models_directory = "trained_models"  # Update to match the directory where models are saved

    predictions = predict_future_prices(user_ticker, user_days, models_directory)
    
    # Print the predicted prices
    for date, price in predictions.items():
        print(f"Date: {date}, Predicted Price: {price}")


Enter a stock ticker (e.g., AAPL, MSFT, ORCL): MSFT
Enter the number of future days for prediction: 10
Date: 2023-09-30, Predicted Price: 314.70822768019565
Date: 2023-10-01, Predicted Price: 315.29653602540367
Date: 2023-10-02, Predicted Price: 316.4124648104292
Date: 2023-10-03, Predicted Price: 315.9962749781248
Date: 2023-10-04, Predicted Price: 314.8325335156399
Date: 2023-10-05, Predicted Price: 315.07165073747245
Date: 2023-10-06, Predicted Price: 316.2564521548709
Date: 2023-10-07, Predicted Price: 316.195347589643
Date: 2023-10-08, Predicted Price: 315.0160496971553
Date: 2023-10-09, Predicted Price: 314.90213840924724
